In [3]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from time import sleep
import numpy as np
import cvzone
from pynput.keyboard import Controller

In [4]:
cap = cv2.VideoCapture(0)
cap.set(3, 1280) 
cap.set(4, 720)
detector = HandDetector(detectionCon=0.8,maxHands=2)
resultText=""
keyboard=Controller()

def drawAll(frame,buttonList):
    img=np.zeros_like(frame,np.uint8)
    for button in buttonList:
        x,y=button.pos
        w,h=button.size
        cvzone.cornerRect(img,(x,y,w,h),l=30,t=5,rt=0,colorC=(255,255,255),colorR=(255,255,255))
        #cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,125),cv2.FILLED)
        cv2.putText(frame,button.text,(x+15,y+37),cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),2)
    
    out=frame.copy()
    alpha=0.2
    mask=img.astype(bool)
    out[mask]=cv2.addWeighted(frame,alpha,img,1-alpha,0)[mask]
    return out

class Button():
    
    def __init__(self,pos,text,size=[50,50]):
        self.pos=pos
        self.size=size
        self.text=text
    
buttonList=[]
keys=[["Q","W","E","R","T","Y","U"],
      ["I","O","P","A","S","D","F"],
      ["G","H","J","K","L","Z","X"],
      ["C","V","B","N","M",",","."]]
for i in range(len(keys)):
        for j,key in enumerate(keys[i]):
            buttonList.append(Button([60*j+100,70*i+10],key))

while True:
    ret,frame=cap.read()
    position,frame=detector.findHands(frame)
    frame=drawAll(frame,buttonList)
    if position:
        if len(position[0]['lmList']) >= 8:
            index_position = tuple(position[0]['lmList'][7])
            middle_position = tuple(position[0]['lmList'][11])
            for button in buttonList:
                x,y=button.pos
                w,h=button.size
                if x<index_position[0]<x+w and y<index_position[1]<y+h:
                    cv2.rectangle(frame,button.pos,(x+w,y+h),(0,0,0),cv2.FILLED)
                    cv2.putText(frame,button.text,(x+15,y+37),cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),2)
                    l, _, _ = detector.findDistance(index_position[:2],middle_position[:2],frame)
                    if l<22:
                        keyboard.press(button.text)
                        cv2.rectangle(frame,button.pos,(x+w,y+h),(0,255,0),cv2.FILLED)
                        cv2.putText(frame,button.text,(x+15,y+37),cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),2)
                        sleep(0.5)
    
    cv2.imshow("frame",frame)
    k=cv2.waitKey(1)
    if k==27:
        break

cap.release()
cv2.destroyAllWindows()